In [1]:
## In questo notebook l'idea è di mettere insieme i diversi dati in modo da avere una matrice unica
import pandas as pd
import plotly.express as px
import json
from statistics import mode
with open("data/BergamoGiusto.geojson") as f:
    paesi = json.load(f)

comuni=pd.read_csv("data/ComuniFinal.csv")
file="https://docs.google.com/spreadsheets/d/166me1IufiXYAHhRL6sgrXJxJTE1fVXQ-RegqU1LqEFY/export?format=csv"
SHEET_ID = '166me1IufiXYAHhRL6sgrXJxJTE1fVXQ-RegqU1LqEFY'
list_of_df=[]
for i in range(1,15):
    SHEET_NAME = 'Risposte%20del%20modulo%20'+f'{i}'
    url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
    df=pd.read_csv(url)
    new=comuni.merge(df, left_on='Comune', right_on='Di che paese sei?', how='outer')
    final=new.groupby('Comune')[new.columns[3]].agg(lambda x: mode(x)).reset_index()
    final.head(20)
    final=final.fillna("Nessun Dato")

    final.columns[1]
    fig = px.choropleth_mapbox(final, geojson=paesi, locations="Comune", color=final.columns[1], featureidkey="properties.name", 
                            mapbox_style="carto-positron",
                            color_discrete_map={
                    "Nessun Dato": "grey"},center={'lat':45.742,'lon':9.5688})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    nome=final.columns[1][10:-1]
    fig.write_json('static/plotly/'+nome+".json")